In [7]:

import pandas as pd
from sklearn.model_selection import train_test_split
df=pd.read_csv('../data/train.csv')
df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train, test = train_test_split(df,test_size=0.15,random_state = 42)
X_train = train.comment_text
X_test = test.comment_text

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


stop_words = set(stopwords.words('english'))
# Define a pipeline combining a text feature extractor with multi lablel classifier
NBmodelTfdiffpipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

RFmodelTfdiffpipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(RandomForestClassifier(random_state = 1,n_estimators = 10))),
            ])

LRmodelTfdiffpipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(random_state = 1))),
            ])
             

In [10]:
pred_df = pd.DataFrame() 
models = {"NBmodelTfdiff": NBmodelTfdiffpipeline,"RFmodelTfdiff":RFmodelTfdiffpipeline,"LRmodelTfdiff":LRmodelTfdiffpipeline }
def trainModel(dataframe):
    

    
    for model in models:
        scores = []
        pipeline = models[model]
        for category in categories:
            print('... Processing {}'.format(category))
            # train the model using X_dtm & y
            pipeline.fit(X_train, train[category])
            # compute the testing accuracy
            prediction = pipeline.predict(X_test)
            accuracy = accuracy_score(test[category], prediction)
            scores.append(accuracy)
        df2 = {'Model':model,'Accuracy': sum(scores) / len(scores)} 
        dataframe = dataframe.append(df2, ignore_index = True) 
        scores = []
    return dataframe

pred_df = trainModel(pred_df)
pred_df

    
    

... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate
... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate
... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate


,Accuracy,Model
0,0.967810,NBmodelTfdiff
1,0.978192,RFmodelTfdiff
2,0.980935,LRmodelTfdiff


sorteddf = pred_df['Accuracy'].argmax()
dataframe.iloc[sorteddf]

In [11]:
maxrow = pred_df['Accuracy'].argmax() 
pred_df.iloc[maxrow]

Accuracy         0.980935
Model       LRmodelTfdiff
Name: 2, dtype: object

In [13]:
import pandas as pd
testdf = pd.read_csv('../data/test.csv')
testlabelsdf = pd.read_csv('../data/test_labels.csv')
testdata = pd.merge(testdf, testlabelsdf, on='id', how='outer')

testdata.drop(testdata[testdata['toxic'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['severe_toxic'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['obscene'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['threat'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['insult'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['identity_hate'] == -1].index, inplace = True)
testdata

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [15]:


categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# train, test = train_test_split(df,test_size=0.15,random_state = 42)

models = {"NBmodelTfdiff": NBmodelTfdiffpipeline,"RFmodelTfdiff":RFmodelTfdiffpipeline,"LRmodelTfdiff":LRmodelTfdiffpipeline }
pred_df = pd.DataFrame() 

def predict(model):
    modelAccuracy = []
    for category in categories:
        topmodel = models[model]
        topmodel.fit(X_train, train[category])
        prediction =  topmodel.predict(testdata['comment_text'])
        accuracy = accuracy_score(testdata[category], prediction)
        modelAccuracy.append(accuracy)
        print('Category '+ category)
        print(accuracy)
    print('Accuracy on the whol Test Data:')
    print(sum(modelAccuracy) / len(modelAccuracy))
predict('LRmodelTfdiff')

Category toxic
0.9377754853230799
Category severe_toxic
0.993263309262559
Category obscene
0.9663321766857357
Category threat
0.9967176216824534
Category insult
0.9638938385069867
Category identity_hate
0.99015286504736
Accuracy on the whol Test Data:
0.9746892160846957


In [ ]:
Citation : "https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5"